In [257]:
import folium
import pandas as pd
import sys
import json
import csv 
import os
from IPython.display import HTML
import math
import time
from selenium import webdriver

In [263]:
base_loc = os.path.join('C:\\','Users','laslo','OneDrive','Documents','Maria','MapAnimation')
    
raw_loc = os.path.join('..','EconomicPredictions','Maps','RawData')
clean_loc = os.path.join('..','EconomicPredictions','Maps','static','assets','geojson')
map_html = os.path.join(base_loc,'map_html')
map_png = os.path.join(base_loc,'map_png')
map_html

'C:\\Users\\laslo\\OneDrive\\Documents\\Maria\\MapAnimation\\map_html'

In [135]:
xlsfile = os.path.join(raw_loc,'GeoFRED_Unemployment_Rate_by_County_Percent2.xls')
xl = pd.ExcelFile(xlsfile)
df0 = xl.parse('Sheet0')
df0.drop(columns =['Series ID'], inplace = True) 

df1 = xl.parse('Sheet1')
df1.drop(columns =['Series ID','Region Name'], inplace = True) 

df2 = xl.parse('Sheet2')
df2.drop(columns =['Series ID','Region Name'], inplace = True) 

unemp = pd.merge(df0,pd.merge(df1,df2,on='Region Code'),on='Region Code')
cols_in = unemp.columns
print('Original Column Names:')
print(cols_in)
cols_rename = []

for col in cols_in:
    
    if col == 'Region Code':
        cols_rename.append('FIPS')
    elif col in ('Series ID','Region Name') or col[0:3] in ('198','197'):
        cols_rename.append('DROP')
    elif "January" in col:
        cols_rename.append(f"Unemp_{col[0:4]}01")
    elif "February" in col:
        cols_rename.append(f"Unemp_{col[0:4]}02")
    elif "March" in col:
        cols_rename.append(f"Unemp_{col[0:4]}03")
    elif "April" in col:
        cols_rename.append(f"Unemp_{col[0:4]}04")
    elif "May" in col:
        cols_rename.append(f"Unemp_{col[0:4]}05")
    elif "June" in col:
        cols_rename.append(f"Unemp_{col[0:4]}06")
    elif "July" in col:
        cols_rename.append(f"Unemp_{col[0:4]}07")
    elif "August" in col:
        cols_rename.append(f"Unemp_{col[0:4]}08")
    elif "September" in col:
        cols_rename.append(f"Unemp_{col[0:4]}09")
    elif "October" in col:
        cols_rename.append(f"Unemp_{col[0:4]}10")
    elif "November" in col:
        cols_rename.append(f"Unemp_{col[0:4]}11")
    elif "December" in col:
        cols_rename.append(f"Unemp_{col[0:4]}12")
    else:
        cols_rename.append(col)

print('Renamed Column Names:')
print(cols_rename)

# Reset the column names
unemp.columns = cols_rename

# Get rid of dropped columns
unemp.drop(columns =['DROP'], inplace = True) 
unemp["In Unemp"] = 1


# Make sure the fips codes have leading zeros.
for obs in range(0,len(unemp)):
    if len(str(unemp.loc[obs,'FIPS'])) == 4:
        unemp.loc[obs,'FIPS'] = f"0{unemp.loc[obs,'FIPS']}"
    else:
        unemp.loc[obs,'FIPS'] = f"{unemp.loc[obs,'FIPS']}"
        
    # 46113 was renamed from Shannon County to Oglala Lakota County, SD in May 2015.  Oglala should have FIPS 46102 but
    # 46102 is not found on the big geojson file so we reset 46102 to 46113.
    if str(unemp.loc[obs,'FIPS']) == '46102':
        unemp.loc[obs,'FIPS'] = '46113'

unemp.head()

Original Column Names:
Index(['Region Name', 'Region Code', '1976 January', '1976 February',
       '1976 March', '1976 April', '1976 May', '1976 June', '1976 July',
       '1976 August',
       ...
       '2018 October', '2018 November', '2018 December', '2019 January',
       '2019 February', '2019 March', '2019 April', '2019 May', '2019 June',
       '2019 July'],
      dtype='object', length=525)
Renamed Column Names:
['DROP', 'FIPS', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP

,FIPS,Unemp_199001,Unemp_199002,Unemp_199003,Unemp_199004,Unemp_199005,Unemp_199006,Unemp_199007,Unemp_199008,Unemp_199009,...,Unemp_201811,Unemp_201812,Unemp_201901,Unemp_201902,Unemp_201903,Unemp_201904,Unemp_201905,Unemp_201906,Unemp_201907,In Unemp
0,45001,7.0,6.8,6.8,5.5,5.5,6.4,9.3,6.4,5.7,...,3.5,3.8,4.4,3.6,3.8,3.3,3.9,4.2,3.8,1
1,22001,5.7,5.1,5.1,4.8,5.6,7.1,9.6,8.1,7.3,...,4.8,4.8,5.7,4.8,4.2,3.9,4.6,5.8,5.5,1
2,51001,8.3,8.1,6.7,5.4,4.4,3.7,3.1,5.2,5.0,...,3.1,3.8,5.1,4.8,4.0,3.3,3.3,3.2,3.1,1
3,16001,4.5,4.5,3.9,3.3,3.2,3.6,3.2,3.2,3.6,...,2.3,2.4,3.3,2.9,2.7,2.2,2.1,2.4,2.5,1
4,19001,4.0,3.7,3.3,2.5,1.6,1.9,1.9,2.1,1.8,...,1.7,2.1,2.9,2.9,2.7,2.0,1.9,2.1,2.1,1


In [136]:
# Start with 201001
geo_file = os.path.join(clean_loc,'FinalGeoFile2000.json')
data_to_plot = unemp[['FIPS','Unemp_200001']]

In [191]:
# select unemployment from 200001 for each county
umax = math.ceil(unemp['Unemp_200001'].max()) + 1
legend_values = []
for x in range(umax):
    legend_values.append(x)

In [196]:
# Setup a folium map at a high-level zoom
umap_200001 = folium.Map(location=[39.82, -98.58], zoom_start=3.5)

# choropleth maps bind Pandas Data Frames and json geometries.
#This allows us to quickly visualize data combinations
umap_200001.choropleth(geo_data=geo_file, data=data_to_plot,
             columns=['FIPS', 'Unemp_200001'],
             key_on='feature.properties.FIPS',
             fill_color='YlOrRd', 
             threshold_scale = [0,1,2,3,4,5,6,7,8,umax],
             fill_opacity=0.7, line_opacity=0.5,
             legend_name='test')

In [194]:
umap_200001.save('umap_200001.html')
# Import the Folium interactive html file


from IPython.display import HTML
HTML('<iframe src=umap_200001.html width=750 height=450></iframe>')

In [149]:
unemployment_dict = data_to_plot.set_index('FIPS')['Unemp_200001']
unemployment_dict['22001']
#unemployment_dict['02201']


6.1

In [288]:
def unemp_colors2(feature):
    
    try: 
        test_value = unemployment_dict[feature['properties']['FIPS']]
    except:
        test_value = -1
        
    #print(test_value)
    
    """Maps low values to green and hugh values to red."""
    if test_value > 9:
        return '#a50026' 
    elif test_value > 8:
        return '#d73027'
    elif test_value > 7:
        return '#f46d43'
    elif test_value > 6:
        return  '#fdae61' 
    elif test_value > 5:
        return '#fee08b'
    elif test_value > 4:
        return '#d9ef8b'
    elif test_value > 3:
        return '#a6d96a'
    elif test_value > 2:
        return '#66bd63'
    elif test_value > 1:
        return '#1a9850' 
    elif test_value > 0:
        return '#006837'
    else:
        return "#lightgray"
    

In [290]:

m2 = folium.Map([43,-100], tiles='cartodbpositron', zoom_start=4.25)

folium.GeoJson(
    geo_file,
    style_function=lambda feature: {
        'fillColor': unemp_colors2(feature),
        'fillOpacity' : '0.9',
        'color' : 'black',
        'weight' : 1
        },   
    highlight_function=lambda x: {'weight':2, 
                                  'fillOpacity':1},    
    tooltip=folium.features.GeoJsonTooltip(
            fields=['CountyName','StateAbbr','Unemp_200001'],
            aliases=['County Name:', 'State:','Unemployment Rate January 2000:'])      
    ).add_to(m2)

save_html = os.path.join(map_html,'umap_200001.html')
save_png = os.path.join(map_png,'umap_200001.png')
m2.save(save_html)
# Import the Folium interactive html file

HTML(f'<iframe src={save_html} width=750 height=450></iframe>')

In [291]:
legend_html = '''
     <div style="position: fixed; 
                 bottom: 0;
                 right: 0;
                 padding: 6px 8px;
                 width: 20%;
                 font: 12px Arial, Helvetica, sans-serif;
                 font-weight: bold;
                 background: #8d8a8d;
                 border-radius: 5px;
                 box-shadow: 0 0 15px rgba(0, 0, 0, 0.2);
                 line-height: 18px;
                 color: 'black';">

     <ul>
     <i style="background: #a50026"> &nbsp &nbsp</i> 9+ <br>
     <i style="background: #d73027"> &nbsp &nbsp</i> 8 - 9<br>
     <i style="background: #f46d43"> &nbsp &nbsp</i> 7 - 8<br>
     <i style="background: #fdae61"> &nbsp &nbsp</i> 6 - 7<br>
     <i style="background: #fee08b"> &nbsp &nbsp</i> 5 - 6<br>
     <i style="background: #d9ef8b"> &nbsp &nbsp</i> 4 - 5<br>
     <i style="background: #a6d96a"> &nbsp &nbsp</i> 3 - 4<br>
     <i style="background: #66bd63"> &nbsp &nbsp</i> 2 - 3<br>
     <i style="background: #1a9850"> &nbsp &nbsp</i> 1 - 2<br>
     <i style="background: #006837"> &nbsp &nbsp</i> 0 - 1<br>
     </ul>
      </div>
     '''
m2.get_root().html.add_child(folium.Element(legend_html))
m2.save('umap_200001_legend.html')

# Import the Folium interactive html file

#HTML('<iframe src=umap_200001_legend.html width=750 height=450></iframe>')

In [284]:

#Save the map as an HTML file
tmpurl=f'file://{save_html}'.format(path=os.getcwd(),mapfile=save_html)
 
#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(save_html)
#Give the map tiles some time to load
time.sleep(5)
#Grab the screenshot
browser.save_screenshot(save_png)
#Close the browser
browser.quit()

In [306]:
dates = ["1960-04-01T00:00:00.000Z","1961-03-31T00:00:00.000Z"]

In [307]:
import datetime

dates2 = []
for dt in dates:
    
    dt2 = dt[0:10]
    y, m, d = [int(x) for x in dt2.split('-')] 
  
    print(f'Date read: {dt}.')
    print(f'd={d}')
    print(f'm={m}')
    print(f'y={y}')
    
    datef = datetime.date(y, m, d) 
  
    print(f'Date output: {datef}.')
    dates2.append(datef)
    
if dates2[0] > dates2[1]:
    print(f"Date 1 ({dates2[0]}) greater than date 2 ({dates2[1]})")
    
elif dates2[0] < dates2[1]:
    print(f"Date 1 ({dates2[0]}) less than date 2 ({dates2[1]})")
elif dates2[0] == dates2[1]:
    print(f"Date 1 ({dates2[0]}) same as date 2 ({dates2[1]})")

Date read: 1960-04-01T00:00:00.000Z.
d=1
m=4
y=1960
Date output: 1960-04-01.
Date read: 1961-03-31T00:00:00.000Z.
d=31
m=3
y=1961
Date output: 1961-03-31.
Date 1 (1960-04-01) less than date 2 (1961-03-31)


In [313]:
timepoint='1990-10-23'

date2check = datetime.date(int(timepoint[0:4]), int(timepoint[5:7]), int(timepoint[9:11]))
print(date2check)

1990-10-03


In [314]:
place = timepoint.find('-')
place

4

In [316]:
test = "1961-03-31T00:00:00.000Z"
test2 = test[test.find('-')+1:]
test2

'03-31T00:00:00.000Z'

In [319]:
# Create a dictionary to hold state and county fips codes
# fipsdict has the state and county associated with each fips code
fipsdict = {}

# Go through the list of counties in our data and make sure each is 
# represented in the fipsdict. This will allow us to have FIPS, STATEFP,
# and COUNTYFP on each geojson file.
for row, rowvals in unemp.iterrows():
    
    # pull information from dataframe into temporary variables
    FIPS = str(rowvals[0])
    STATEFP = str(FIPS)[0:2]
    COUNTYFP = str(FIPS)[2:5]
    
    # If we have not yet seen this FIPS code, add it to the dictionary
    if FIPS not in fipsdict:
        fipsdict[FIPS]={}
 
    # Append the data for this county to its entry in the dictionary
    fipsdict[FIPS]={
        "STATEFP": STATEFP,
        "COUNTYFP": COUNTYFP}
    
featureData = fipsdict.get('22001', {})
featureData

{'STATEFP': '22', 'COUNTYFP': '001'}